# NLTK

Fist, install some packs

```
pip3 install -U spacy  
```

```
python3 -m spacy download en
```

```
pip install pyldavis
```

ref: https://datascienceplus.com/topic-modeling-in-python-with-nltk-and-gensim/

In [1]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [2]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /Users/zireael/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# nltk.download('stopwords')
# en_stop = set(nltk.corpus.stopwords.words('english'))

from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))


new_stopwords = ['|'] #no neet since the file was updated. more info to came
new_stopwords_list = stop_words.union(new_stopwords)

print(new_stopwords_list)

{'d', 'does', "wasn't", 'most', 'no', 'aren', "she's", 'wouldn', 'and', 'what', 'the', "isn't", 'yourself', 'we', 'did', 'won', 'under', 'more', 'over', 'has', 'from', "doesn't", 'that', 'these', 'shouldn', 'who', 'if', 'be', 'to', 'for', 'y', 'didn', 'hasn', 'off', 'how', 'ours', 'haven', "won't", 'him', 'while', 'can', 'those', 'our', 'above', 'with', 'few', 'are', 'such', "didn't", "aren't", 'too', 'i', 'up', 'them', 'hadn', 'further', 'isn', 'its', 'so', 'an', 'she', 'some', 'itself', 'you', 'll', "that'll", 'hers', 'below', 're', 'or', "shan't", 'ma', 'they', 'being', "hadn't", 'against', 'on', 'themselves', "wouldn't", 'once', 'shan', 'weren', 'whom', 'into', 'a', 'until', 'ain', 'by', 's', 'out', 'myself', 'needn', 'ourselves', "mustn't", "you're", 'this', 'but', 'doesn', 'were', 'mightn', 'all', 'was', 'is', 'as', "weren't", 'her', 'because', 'own', 'just', "you'll", 'should', 'after', 'very', "you've", 'theirs', "shouldn't", 'he', 'their', 'at', '|', 'then', 'have', 'don', 'wh

In [4]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in new_stopwords]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

Topic model file is the database only replacing some values so it will have a aproximate behaviour as a textual document, as having space between values(representing words) and replacing every space with a underscore so the 'palavara composta?' dont be lost without its context

In [5]:
import random
text_data = []

with open('data/treated_data/topic-model.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['52686', 'p-(2,2,2-trichloro-1-hydroxyethyl)phosphonic_acid_dimethyl_ester', 'not_reported', 'mammal', 'mus_musculus', 'house_mouse', 'not_reported', 'not_reported', 'not_reported', 'gavage', 'mortality', 'mortality', 'not_reported', 'not_reported', 'active_ingredient', 'kg_org', 'day(s', 'haley', 't.j.,_j.h._farmer,_j.r._harmon,_and_k.l._dooley', '35203', 'estimation_of_the_ld1_and_extrapolation_of_the_ld0.1_for_five_organophosphate_pesticides', 'arch._toxicol.34:103']
['56531', "4,4'-[(1e)-1,2-diethyl-1,2-ethenediyl]bisphenol", 'technical_grade,_technical_product,_technical_formulation', 'mammal', 'peromyscus_maniculatus', 'deer_mouse', 'not_reported', 'not_reported', 'not_reported', 'oral_via_capsule', 'mortality', 'mortality', 'not_reported', 'not_reported', 'active_ingredient', 'day(s', 'schafer', 'e.w.,jr', ',_and_w.a.,jr._bowles', '35426', 'acute_oral_toxicity_and_repellency_of_933_chemicals_to_house_and_deer_mice', 'arch._environ._contam._toxicol.14(1):_111']
['60571', '1ar,2r

['7778394', 'arsenic_acid__(h3aso4', 'not_coded', 'mammal', 'rattus_rattus', 'black_rat', 'not_coded', 'not_coded', 'not_coded', 'mortality', 'mortality', 'not_coded', 'not_coded', 'not_coded', 'day(s', 'u.s._environmental_protection_agency', 'pesticide_ecotoxicity_database_(formerly:__environmental_effects_database_(eedb', 'environmental_fate_and_effects_division,_u.s.epa,_washington,_d.c.']
['14769734', 's)-2,3,5,6-tetrahydro-6-phenyl', 'imidazo[2,1-b]thiazole', 'not_reported', 'mammal', 'mus_musculus', 'house_mouse', 'young', 'not_reported', 'no_substrate', 'mortality', 'mortality', 'not_reported', 'unmeasured', 'formulation', 'kg_bdwt', 'day(s', 'brody', 'g.,_and_t.e._elward', '101241', 'comparative_activity_of_29_known_anthelmintics_under_standarized_drug', 'diet_and_gavage_medication_regimens_against_four_helminth_species_in_mice', 'j._parasitol.57(5):_1068']
['52918635', '1r,3r)-3-(2,2-dibromoethenyl)-2,2-dimethylcyclopropanecarboxylic_acid_(s)cyano(3-phenoxyphenyl)methyl_ester'

In [6]:
from gensim import corpora

dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [7]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.113*"not_reported" + 0.073*"mortality" + 0.045*"day(s" + 0.037*"unmeasured"')
(1, '0.112*"not_coded" + 0.034*"mortality" + 0.019*"rattus_rattus" + 0.019*"environmental_fate_and_effects_division,_u.s.epa,_washington,_d.c."')
(2, '0.167*"not_reported" + 0.077*"mortality" + 0.046*"day(s" + 0.026*"mammal"')
(3, '0.004*"not_reported" + 0.004*"mortality" + 0.004*"day(s" + 0.004*"mammal"')
(4, '0.067*"not_reported" + 0.045*"mortality" + 0.024*"no_substrate" + 0.024*"active_ingredient"')


In [8]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim

lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
